In [5]:
import os
import glob
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datasets

In [6]:
#
path_parent = r'/Users/vijetadeshpande/Downloads/UMass Lowell - Courses/Spring 2022/Foundations in Digital Health/FDigital_Health_Spring_2022/Project_final'
path_data = os.path.join(path_parent, r'Data/Track2_SubtaskA/Annotations')
path_train = os.path.join(path_data, r'train/mimic')
path_val = os.path.join(path_data, r'dev/mimic')


In [7]:
# first read all files
files_train = []
files_val = []

files_train += glob.glob(path_train + "/*.txt")
files_train += glob.glob(path_train + "/*.ann")

files_val += glob.glob(path_val + "/*.txt")
files_val += glob.glob(path_val + "/*.ann")

#
print(len(files_train))
print(len(files_val))

2632
376


In [8]:
# first load similar dataset to see the characteristics of the NER and RE data

sem_eval = datasets.load_dataset('sem_eval_2010_task_8')
doc_red = datasets.load_dataset('xiaobendanyn/tacred')

Using the latest cached version of the module from /Users/vijetadeshpande/.cache/huggingface/modules/datasets_modules/datasets/sem_eval_2010_task_8/8545d1995bbbade386acf5c4e2bef5589d8387ae0a93356407dfb54cdb234416 (last modified on Sat Apr 23 12:30:09 2022) since it couldn't be found locally at sem_eval_2010_task_8., or remotely on the Hugging Face Hub.
Using custom data configuration default
Reusing dataset sem_eval2010_task8 (/Users/vijetadeshpande/.cache/huggingface/datasets/sem_eval2010_task8/default/1.0.0/8545d1995bbbade386acf5c4e2bef5589d8387ae0a93356407dfb54cdb234416)


  0%|          | 0/2 [00:00<?, ?it/s]

ConnectionError: Couldn't reach 'xiaobendanyn/tacred' on the Hub (ConnectionError)

In [9]:
#
#print(sem_eval['train'][200])
print(doc_red['train'][200])

NameError: name 'doc_red' is not defined

In [10]:
# read the data in files
data_train = {'x': [], 'y': []}

for idx, file in enumerate(files_train):
    if file[-3:] == 'ann':
        break
    
    #
    with open(file, 'r') as f:
        x_ = f.read()
    with open(file[:-3]+'ann', 'r') as f:
        y_ = f.read()
    
    print("="*5)
    print(x_)
    print(y_)
    
    if idx > 5:
        break

=====
Social History:

=====
Social History:
The patient has smoked for 27 years, an unspecified amount. She
does not currently drink but has a long history of ethanol
abuse.  She is disabled.
T1	Tobacco 32 38	smoked
T2	Duration 39 51	for 27 years
T3	StatusTime 28 38	has smoked
T4	Alcohol 131 138;139 144	ethanol abuse
T5	StatusTime 80 127	does not currently drink but has a long history
E1	Tobacco:T1 Status:T3 Duration:T2
E2	Alcohol:T4 Status:T5
A1	StatusTimeVal T3 current
A2	StatusTimeVal T5 past
T6	Employment 154 162	disabled
E3	Employment:T6 Status:T7
T7	StatusEmploy 154 162	disabled
A3	StatusEmployVal T7 on_disability

=====
Social History:
Lives with:alone, Can stay at nephews or brothers if needed
Contact:[**Name (NI) **] (Brother) Phone #
Occupation:inspector at R&D Engineering
Cigarettes: Smoked no [] yes [x] last cigarette [**2100-8-11**] Hx:<1ppd
x 40 years, Has quit in past for up to 3 yrs
Other Tobacco use:denies
ETOH: Recovered alcoholic. No ETOH in 22 years
Illicit drug us

In [11]:
# from above exmaples it is clear that there are multiple overlapping entities
# therefore the task is not flat-NER, it is nested NER.
# if we formulate the problem as MRC then we will need to come-up with questions for each kind of entity and respective attributes

# sample query
query_template = {
    'status': '[ENT-TYPE] status?',
    'duration': '[ENT-TYPE] duration?',
    'history': '[ENT-TYPE] history?',
    'type': '[ENT-TYPE]?',
    'amount': '[ENT-TYPE] amount?',
    'frequency': '[ENT-TYPE] frequency?',
    'method': '[ENT-TYPE] method?',
}

# example of question
query_dict = {
    'tobacco': {},
    'alcohol': {},
    'drug': {},
    'employment': {},
    'living status': {},
    'insurance': {},    
    'sexual orientation': {},    
    'gender identity': {},
    'country of origin': {},
    'race': {},
    'physical activity': {},
    'environmental exposure': {},
}

for ent in query_dict:
    for attribute in query_template:
        query_dict[ent][attribute] = query_template[attribute].replace('[ENT-TYPE]', ent)


In [12]:
#
print(query_dict)

{'tobacco': {'status': 'tobacco status?', 'duration': 'tobacco duration?', 'history': 'tobacco history?', 'type': 'tobacco?', 'amount': 'tobacco amount?', 'frequency': 'tobacco frequency?', 'method': 'tobacco method?'}, 'alcohol': {'status': 'alcohol status?', 'duration': 'alcohol duration?', 'history': 'alcohol history?', 'type': 'alcohol?', 'amount': 'alcohol amount?', 'frequency': 'alcohol frequency?', 'method': 'alcohol method?'}, 'drug': {'status': 'drug status?', 'duration': 'drug duration?', 'history': 'drug history?', 'type': 'drug?', 'amount': 'drug amount?', 'frequency': 'drug frequency?', 'method': 'drug method?'}, 'employment': {'status': 'employment status?', 'duration': 'employment duration?', 'history': 'employment history?', 'type': 'employment?', 'amount': 'employment amount?', 'frequency': 'employment frequency?', 'method': 'employment method?'}, 'living status': {'status': 'living status status?', 'duration': 'living status duration?', 'history': 'living status histo

In [38]:
# aux functions
def count_words(list_):
    
    words = 1
    for element in list_:
        if ';' in element and (not element.replace(';', '').isalpha()):
            words += 1
    
    return words

def deal_with_multi_words(list_, num_words):
    list_out = []
    for element in list_:
        if ';' in element and (not element.replace(';', '').isalpha()):
            eles = element.split(';')
            
            # check if sub-spans are next to each other or not
            if (int(eles[0]) - int(eles[1])) > 1:
                print('sub-spans are not next to each other')
                exit()
            
            list_out += eles
        else:
            list_out += [element]
    
    return list_out

def structurize_tags(list_):
    
    total_words = count_words(list_)
    if total_words >= 2:
        list_ = deal_with_multi_words(list_, total_words)
    
    #
    span_start = list_[2]
    span_end = list_[1 + (total_words*2)]
    sub_spans = list_[2:1 + (total_words*2) + 1]
    
    #
    dict_ = {
        'tag index': list_[0],
        'tag type': list_[1],
        'span start': span_start,
        'span end': span_end,
        'sub spans': sub_spans,
        'num words': total_words,
        'text': ' '.join(list_[1 + (total_words*2) + 1: ]),
    }
    
    return dict_

def structurize_relations(list_):
    
    for n_, node in enumerate(list_[1:]):
        if n_ == 0:
            #this is the fixed start node for the current line in the annotation
            continue
        else:
            dict_ = {
                'relation index': list_[0],
                'start node': list_[1].split(':')[-1],
                'start node type': list_[1].split(':')[0],
                'end node': list_[n_].split(':')[-1],
                'end node type': list_[n_].split(':')[0],
            }    
    
    return dict_

def structurize_attributes(list_):
    
    dict_ = {
        'attribute index': list_[0],
        'start node': list_[2],
        'start node type': list_[1],#.replace('Val', ''),
        'text': list_[-1],
    }    
    
    return dict_

def get_positive_query():
    
    return query

In [39]:
def get_annotation_dict(ann):
    #print(ann)
    annot_dict = {
        'tags': [],
        'edges': [],
        'attributes': [],
    }
    
    #
    if ann == '':
        return annot_dict
    
    ann = ann.split('\n')
    for line in ann:
        if line == '':
            continue
        
        #
        line = line.replace(' ', '\t')
        annot = line.split('\t')
        #print(annot)
        
        #
        if annot[0][0] == 'T':
            annot_dict['tags'].append(structurize_tags(annot))
        elif annot[0][0] == 'E':
            annot_dict['edges'].append(structurize_relations(annot))
        elif annot[0][0] == 'A':
            annot_dict['attributes'].append(structurize_attributes(annot))
        else:
            print('Feature other than tag, relation and attribute is present')
            
    return annot_dict

def get_qca_triplets(context, label_dict, query_dict):
    instances = []
    
    # first let's deal with tag type
    for ent in label_dict['tags']:
        query_pos = get_positive_query(ent, query_dict)
        instances.append(
            {
                'context': context,
                'query': query,
                'answer start': ent['span start'],
                'answer end': ent['span end'],
                'answer text': ent['text'],
                'answer entity type': ent['tag type']
            }
        )
    
    return

In [47]:
# now for every instance in the dataset, we need (query, context, answer) triplets

# read the data in files
data_train = []
structured_annotations = []
empty_annot = 0

for subset in [files_train, files_val]:
    for idx, file in enumerate(subset):
        if file[-3:] == 'ann':
            continue

        # key for data instance
        key = file[:-4]

        # read text and annotations
        with open(file, 'r') as f:
            x_ = f.read()
        with open(file[:-3]+'ann', 'r') as f:
            y_ = f.read()

        # if annotations are empty then continue to next file
        if y_ == '':
            empty_annot += 1
            continue

        # structurize the annotations
        y_dict = get_annotation_dict(y_)
        y_dict['instance key'] = key
        structured_annotations.append(y_dict)

        # y_dict has three keys, 'tags', 'edges' and 'attributes'. Each key has list form form of value
        # we will have one query for each  element in 'tags' and 'edges'
        # I DON'T KNOW WHAT TO DO WITH 'attributes'
    

In [48]:
print(len(structured_annotations))
print(empty_annot)

1451
53


In [49]:
# collect unique tag types and unique attribute types
unique_tags = {}
unique_att = {}

for example in structured_annotations:
    for tag in example['tags']:
        tt_ = tag['tag type']
        if tt_ in unique_tags:
            unique_tags[tt_] += 1
        else:
            unique_tags[tt_] = 1
    
    for att in example['attributes']:
        node = att['start node type']
        if node in unique_att:
            unique_att[node] += 1
        else:
            unique_att[node] = 1

In [51]:
unique_att

{'StatusTimeVal': 4617, 'StatusEmployVal': 1072, 'TypeLivingVal': 1076}

In [52]:
unique_tags

{'Tobacco': 1381,
 'Duration': 531,
 'StatusTime': 4617,
 'Alcohol': 1455,
 'Employment': 1072,
 'StatusEmploy': 1072,
 'LivingStatus': 1075,
 'TypeLiving': 1076,
 'Type': 1904,
 'History': 687,
 'Amount': 1007,
 'Drug': 1087,
 'Frequency': 602,
 'Method': 217}